In [1]:
# 1.导入训练集
datas = []
train_path = "./data1/adult.data"
with open(train_path) as f:
    datas = f.readlines()
datas = datas[:-1]    # 去除最后一个空行

In [2]:
# 2. 取出每条数据的每个属性
for i in range(datas.__len__()):
    datas[i] = datas[i].split(',')
    for j in range(datas[i].__len__()):
        datas[i][j] = datas[i][j].strip('\n').strip(' ')    # 去除每个属性前的空格和最后一个属性的换行符‘/n’
print("="*50 + "读取训练集" + "="*50)
print("从 {} 读取到： {} 条数据！".format(train_path, datas.__len__()))


==================================================读取训练集==================================================
从 ./data1/adult.data 读取到： 32561 条数据！


In [3]:
# 3.Remove all the records containing '?' (i.e., missing values). Also, remove the attribute "native-country".
Datas = []
for i in range(datas.__len__()):
    flag = True    # 当前数据可行
    for j in range(datas[i].__len__()):
        if datas[i][j] == '?' or datas[i][j] == 'native-country':  # 如果当前数据含有‘ ？’则剔除这条数据
            flag = False
            break
    if flag:
        Datas.append(datas[i])
print("去除非法数据后，总共有：{} 条数据！".format(Datas.__len__()))


去除非法数据后，总共有：30162 条数据！


In [4]:
# 4. 计算先验概率：收入>50k的概率P(C0)和收入<=50k的概率P(C1)
num_PC_0 = 0
num_PC_1 = 0
for i in range(Datas.__len__()):
    if Datas[i][-1]=='>50K':
        num_PC_0 += 1
    elif Datas[i][-1]=='<=50K':
        num_PC_1 += 1
PC_0 = num_PC_0 / Datas.__len__()
PC_1 = num_PC_1 / Datas.__len__()
print("收入>50k的共有：{} 条记录， 概率为：{}".format(num_PC_0, PC_0))
print("收入<=50k的共有：{} 条记录， 概率为：{}".format(num_PC_1, PC_1))


收入>50k的共有：7508 条记录， 概率为：0.24892248524633645
收入<=50k的共有：22654 条记录， 概率为：0.7510775147536636


In [5]:
# 5.计算每个连续属性的均值和方差
c0_mean = [0, 0, 0, 0, 0, 0]
c1_mean = [0, 0, 0, 0, 0, 0]
c0_d = [0, 0, 0, 0, 0, 0]
c1_d = [0, 0, 0, 0, 0, 0]
for i in range(Datas.__len__()):
    # 计算标签为C0的连续属性的均值
    if Datas[i][-1]=='>50K':
        for idx, j in  enumerate([0, 2, 4, 10, 11, 12]): # 取出连续属性
            c0_mean[idx] += float(Datas[i][j])
    # 计算标签为C1的连续属性的均值
    elif Datas[i][-1]=='<=50K':
        for idx, j in  enumerate([0, 2, 4, 10, 11, 12]): # 取出连续属性
            c1_mean[idx] += float(Datas[i][j])
for i in range(c0_mean.__len__()):
    c0_mean[i] /= num_PC_0
for i in range(c1_mean.__len__()):
    c1_mean[i] /= num_PC_1
print()
print("="*50 + "计算每个连续属性的均值和方差" + "="*50)
print("标签为C0的连续属性的均值为：{}".format(c0_mean))
print("标签为C1的连续属性的均值为：{}".format(c1_mean))


==================================================计算每个连续属性的均值和方差==================================================
标签为C0的连续属性的均值为：[43.95911028236548, 188149.96217368142, 11.606419818859882, 3937.6798082045816, 193.75066595631327, 45.70657964837507]
标签为C1的连续属性的均值为：[36.60806038668668, 190338.64672905448, 9.629116270857244, 148.8938377328507, 53.44800035313852, 39.34859186015715]


In [6]:
for i in range(Datas.__len__()):
    # 计算标签为C0的连续属性的方差
    if Datas[i][-1]=='>50K':
        for idx, j in  enumerate([0, 2, 4, 10, 11, 12]): # 取出连续属性
            c0_d[idx] += (float(Datas[i][j]) - c0_mean[idx]) * (float(Datas[i][j]) - c0_mean[idx])
    # 计算标签为C1的连续属性的方差
    elif Datas[i][-1]=='<=50K':
        for idx, j in  enumerate([0, 2, 4, 10, 11, 12]): # 取出连续属性
            c1_d[idx] += (float(Datas[i][j]) - c1_mean[idx]) * (float(Datas[i][j]) - c1_mean[idx])
for i in range(c0_d.__len__()):
    c0_d[i] /= num_PC_0
for i in range(c1_d.__len__()):
    c1_d[i] /= num_PC_1
print("标签为C0的连续属性的方差为：{}".format(c0_d))
print("标签为C1的连续属性的方差为：{}".format(c1_d))

标签为C0的连续属性的方差为：[105.45131151527511, 10570901483.818233, 5.608680149804254, 206931157.7760209, 351395.3707041601, 115.26752729959043]
标签为C1的连续属性的方差为：[181.28829205321483, 11356949805.692152, 5.825189145745581, 876791.795871855, 96263.38660916287, 142.81469807013272]


In [7]:
# 6.计算每个离散属性的P(xi|Ci)
c0_cnt = [{}, {}, {}, {}, {}, {}, {}, {}]
c1_cnt = [{}, {}, {}, {}, {}, {}, {}, {}]
for i in range(Datas.__len__()):
    # 计算标签为C0的离散属性中每个值的个数
    if Datas[i][-1]=='>50K':
        for idx, j in  enumerate([1, 3, 5, 6, 7, 8, 9, 13]): # 取出离散属性
            if Datas[i][j] not in c0_cnt[idx]:
                c0_cnt[idx][Datas[i][j]] = 1
            else:
                c0_cnt[idx][Datas[i][j]] += 1
    # 计算标签为C1的离散属性中每个值的个数
    elif Datas[i][-1]=='<=50K':
        for idx, j in  enumerate([1, 3, 5, 6, 7, 8, 9, 13]): # 取出离散属性
            if Datas[i][j] not in c1_cnt[idx]:
                c1_cnt[idx][Datas[i][j]] = 1
            else:
                c1_cnt[idx][Datas[i][j]] += 1
# 计算每个离散属性取得对应值的概率P(xi|Ci)
# 保留原先计数，用于后面拉普拉斯改进
c0_prob = []
c1_prob = []
for i in range(c0_cnt.__len__()):
    c0_prob.append(c0_cnt[i].copy())
for i in range(c1_cnt.__len__()):
    c1_prob.append(c1_cnt[i].copy())
for i in range(c0_prob.__len__()):
    for k,v in c0_prob[i].items():
        c0_prob[i][k] = v / num_PC_0
for i in range(c1_prob.__len__()):
    for k,v in c1_prob[i].items():
        c1_prob[i][k] = v / num_PC_1
print()
print("="*50 + "计算每个离散属性的P(xi|Ci)" + "="*50)
# print(c0_cnt)
# print(c1_cnt)
print("标签为C0的离散属性的P(xi|Ci)：{}".format(c0_prob))
print("标签为C1的离散属性的P(xi|Ci)：{}".format(c1_prob))



==================================================计算每个离散属性的P(xi|Ci)==================================================
标签为C0的离散属性的P(xi|Ci)：[{'Self-emp-not-inc': 0.09509856153436334, 'Private': 0.6494405966968567, 'State-gov': 0.04581779435269046, 'Local-gov': 0.0811134789557805, 'Federal-gov': 0.048614810868407035, 'Self-emp-inc': 0.07991475759190197}, {'HS-grad': 0.2153702717101758, 'Masters': 0.12226957911561001, 'Bachelors': 0.28316462440063933, 'Some-college': 0.17794352690463505, 'Doctorate': 0.03729355354288759, 'Prof-school': 0.054075652637187, 'Assoc-voc': 0.04581779435269046, 'Assoc-acdm': 0.034096963239211506, '7th-8th': 0.004661694192860949, '12th': 0.0038625466169419287, '10th': 0.007858284496537027, '11th': 0.007858284496537027, '9th': 0.0033297815663292487, '5th-6th': 0.0015982951518380393, '1st-4th': 0.0007991475759190197}, {'Married-civ-spouse': 0.8522908897176346, 'Never-married': 0.06259989344698988, 'Divorced': 0.06020245071923282, 'Separated': 0.008790623335109217, 

In [8]:
# 7.导入测试集
datas = []
test_path = "./data1/adult.test"
with open(test_path) as f:
    datas = f.readlines()
datas = datas[1:]     # 取出第一行无效数据
datas = datas[:-1]    # 去除最后一个空行
for i in range(datas.__len__()):
    datas[i] = datas[i].split(',')
    for j in range(datas[i].__len__()):
        datas[i][j] = datas[i][j].strip('\n').strip(' ').strip('.')    # 去除每个属性前的空格和最后一个属性的换行符‘/n’和'.'
print()
print("="*50 + "读取测试集" + "="*50)
print("从 {} 读取到： {} 条数据！".format(test_path, datas.__len__()))
Datas = []
for i in range(datas.__len__()):
    flag = True    # 当前数据可行
    for j in range(datas[i].__len__()):
        if datas[i][j] == '?' or datas[i][j] == 'native-country':  # 如果当前数据含有‘ ？’则剔除这条数据
            flag = False
            break
    if flag:
        Datas.append(datas[i])
print("去除非法数据后，总共有：{} 条数据！".format(Datas.__len__()))


==================================================读取测试集==================================================
从 ./data1/adult.test 读取到： 16281 条数据！
去除非法数据后，总共有：15060 条数据！


In [9]:
# 8.测试！
print()
print("="*50 + "开始测试" + "="*50)
print("stat testing ...")
iter = 0    # 记录推理了多少条数据
tp = 0
import math # 这是python的标准库
for i in range(Datas.__len__()):
    PXC_0_continuous = 1
    PXC_0_discrete = 1
    PXC_1_continuous = 1
    PXC_1_discrete = 1
    # 计算连续属性的概率
    for idx, j in  enumerate([0, 2, 4, 10, 11, 12]): # 取出连续属性
        # 对于连续的属性计算高斯概率密度
        # C0
        exponent = math.exp(-(math.pow(float(Datas[i][j])-c0_mean[idx],2))/(2*c0_d[idx]))
        GaussProb = (1/(math.sqrt(2*math.pi)*c0_d[idx]))*exponent
        PXC_0_continuous *= GaussProb
        # C1
        exponent = math.exp(-(math.pow(float(Datas[i][j])-c1_mean[idx],2))/(2*c1_d[idx]))
        GaussProb = (1/(math.sqrt(2*math.pi)*c1_d[idx]))*exponent
        PXC_1_continuous *= GaussProb
    # 计算离散属性的概率
    for idx, j in  enumerate([1, 3, 5, 6, 7, 8, 9, 13]): # 取出离散属性
        # 如果该离散属性的值未在训练集/当前集出现，则使用拉普拉斯改进，更新离散属性的概率P(xi|Ci)
        # ===================== 拉普拉斯改进！！！ =====================#
        # 如果不在c0里，则更新c0_prob
        if Datas[i][j] not in c0_prob[idx]:
            c0_cnt[idx][Datas[i][j]] = 0    # 添加这个新的离散属性的值
            for k, v in c0_cnt[idx].items():
                c0_cnt[idx][k] = v + 1  # 则该离散属性的所有值出现的个数都 +1
            # 重新计算离散属性概率
            for k, v in c0_cnt[idx].items():
                c0_prob[idx][k] = v / sum(c0_cnt[idx].values())
        # 如果不在c1里，则更新c1_prob
        if Datas[i][j] not in c1_prob[idx]:
            c1_cnt[idx][Datas[i][j]] = 0    # 添加这个新的离散属性的值
            for k, v in c1_cnt[idx].items():
                c1_cnt[idx][k] = v + 1  # 则该离散属性的所有值出现的个数都 +1
            # 重新计算离散属性概率
            for k, v in c1_cnt[idx].items():
                c1_prob[idx][k] = v / sum(c1_cnt[idx].values())
        # ===================== 拉普拉斯改进！！！ =====================#
        PXC_0_discrete *= c0_prob[idx][Datas[i][j]]
        PXC_1_discrete *= c1_prob[idx][Datas[i][j]]
    # 最终概率等于所有属性的条件概率相乘(Gaussian Naive Bayes)，再乘上标签概率
    PC0_X = PXC_0_continuous * PXC_0_discrete * PC_0
    PC1_X = PXC_1_continuous * PXC_1_discrete * PC_1
    if PC0_X > PC1_X:
        if Datas[i][-1] == '>50K':
            tp += 1
    else:
        if Datas[i][-1] == '<=50K':
            tp += 1
    if iter == 0 or iter % 6000 == 0:
        print("{:.2f}%".format(iter/Datas.__len__()*100))
    if iter == Datas.__len__() - 1:
        print("100.00%")
    iter += 1
print("done.")


==================================================开始测试==================================================
stat testing ...
0.00%
39.84%
79.68%
100.00%
done.


In [10]:
# 9.评估！
print()
print("="*50 + "评估结果" + "="*50)
print("一共有：{} 条测试数据".format(Datas.__len__()))
print("预测正确的有：{} 条数据".format(tp))
precision = tp/Datas.__len__()*100
print("测试集的准确率为：{:.4f}%".format(precision))


==================================================评估结果==================================================
一共有：15060 条测试数据
预测正确的有：12122 条数据
测试集的准确率为：80.4914%
